# Global Imports

In [ ]:
import os
import pprint
import sys
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec

from utils_002 import gen_xyz, load_cfg


def read_simlog(simlog_file: Path) -> Dict[str, np.ndarray]:
    """
    Reads a simulation log file and returns its data as a dictionary
    of NumPy arrays.

    Args:
        simlog_file: The path to the 'SimLog.txt' file.

    Returns:
        A dictionary containing simulation data.
    """
    try:
        lines = simlog_file.read_text(encoding="utf-8").splitlines()
    except FileNotFoundError:
        print(f"Error: The file {simlog_file} was not found.")
        return {}

    simlog_data = []
    for line in lines:
        stripped_line = line.strip()
        try:
            data_points = [float(val) for val in stripped_line.split(":")]
            simlog_data.append(np.array(data_points))
        except (ValueError, IndexError) as e:
            # print(
            #     f"Warning: Skipping malformed line in {simlog_file}: '{line.strip()}' - {e}"
            # )
            simlog_data.append(np.array([]))

    keys = [
        "n_gas",
        "n_crystal",
        "concentration",
        "delta_gibbs",
        "energy_change",
        "crystal_sx",
        "crystal_sy",
        "crystal_sz",
        "tri",
    ]

    if len(simlog_data) != len(keys):
        print(
            f"Warning: Mismatch between expected data lines ({len(keys)}) and actual lines ({len(simlog_data)}) in {simlog_file}."
        )
        return {}

    return dict(zip(keys, simlog_data))


def process_directory(dir_path: Path) -> Dict[str, Dict[str, np.ndarray]]:
    """
    Processes a directory containing simulation results,
    finding and parsing 'InitSettings.ini' and 'SimLog.txt' files.

    Args:
        dir_path: The root directory path to process.

    Returns:
        A dictionary where keys are sub-directory IDs and values are
        the parsed simulation data.
    """
    cfg_file = dir_path / "InitSettings.ini"
    main_simlog_file = dir_path / "SimLog.txt"
    if not cfg_file.exists():
        raise FileNotFoundError(f"Missing InitSettings.ini in {dir_path}")
    if not main_simlog_file.exists():
        raise FileNotFoundError(f"Missing SimLog.txt in {dir_path}")

    cfg = load_cfg(
        path_to_config=cfg_file,
        com_line="/////////////////////////////// | GENERAL INFO | ///////////////////////////////",
    )

    # sx, sy, sz = cfg["Sx"], cfg["Sy"], cfg["Sz"]
    # x, y, z = gen_xyz(sx, sy, sz, mode=2)

    try:
        main_id = dir_path.name.split("_", 1)[0]
    except IndexError:
        print(f"Warning: Could not extract ID from {dir_path.name}")
        main_id = dir_path.name

    items_data = {
        "cfg": cfg,
        "main": {"ID": main_id, "data": read_simlog(main_simlog_file)},
        "sub": {},
    }
    for sub_dir_path in dir_path.glob("**/"):
        states_file = sub_dir_path / "TimeStates.txt"
        simlog_file = sub_dir_path / "SimLog.txt"
        if states_file.exists() and simlog_file.exists():
            print(f"--> Processing: {sub_dir_path}")

            try:
                sub_id = sub_dir_path.name.split("_", 1)[0]
            except IndexError:
                print(
                    f"Warning: Could not extract IDs from {dir_path.name} or {sub_dir_path.name}"
                )
                sub_id = sub_dir_path.name

            items_data["sub"][sub_id] = {"ID": sub_id, "data": read_simlog(simlog_file)}

    return items_data

In [ ]:
BASE_PATH = Path(  #
    r"1754236522824104_Mode2_N100_X50Y200Z50_T3e2_C9.58767e-8_Nt1e11_Pb0.05"
    #
)

try:
    print(f"Processing base directory: {BASE_PATH}")
    all_data = process_directory(BASE_PATH)

except FileNotFoundError as e:
    print(f"Error: {e}")
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    sys.exit(1)

In [ ]:
# pprint.pprint(all_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)

for name, item in all_data["sub"].items():
    ID, DATA = item["ID"], item["data"]
    ax.plot(DATA["tri"], DATA["crystal_sy"])

ax.grid(True)
plt.show()
plt.close()

In [ ]:
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 2.5,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

In [ ]:
MAX_SUB_LEN = max(
    [item["data"]["crystal_sy"].size for name, item in all_data["sub"].items()]
)

base = np.full(MAX_SUB_LEN, None)

In [ ]:
sub = {}

for k, v in all_data["sub"].items():
    ID, DATA = v["ID"], v["data"]
    # print(DATA.keys())

    for k1, v1 in DATA.items():
        v1_0 = v1[:-1]
        base.fill(None)
        base[: v1_0.size] = v1_0[:]
        if (base != None).sum() == 0:
            tmp = np.array([])
        else:
            tmp = base.copy()
        DATA[k1] = tmp
        # print(ID, k1, tmp)

    sub[k] = {"ID": ID, "data": DATA.copy()}

all_data["sub"] = sub.copy()
del k, v, k1, v1, ID, DATA, tmp, sub

In [ ]:
# pprint.pprint(all_data)

In [ ]:
CFG, MAIN, SUB = all_data["cfg"].copy(), all_data["main"].copy(), all_data["sub"].copy()
del all_data

In [ ]:
keys = set()
for item_keys in map(lambda item: set(item["data"].keys()), SUB.values()):
    keys.update(item_keys)
print(keys)

SUB1 = {k: np.vstack([*map(lambda item: item["data"][k], SUB.values())]) for k in keys}
SUB2 = {k: v.T for (k, v) in SUB1.items()}

In [ ]:
path_to_dir = Path(  #
    r"1754236522824104_Mode2_N100_X50Y200Z50_T3e2_C9.58767e-8_Nt1e11_Pb0.05\Graphs"
    #
)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = SUB2["tri"]
Y = SUB2["energy_change"]

XY = []

for (t1, t2) in zip(X, Y):
    mask = (t1 != None) & (t2 != None)
    if True in mask:
        XY.append((np.mean(t1[mask]), np.mean(t2[mask])))

X, Y = np.array(XY).T

datasets = [
    ("Data", X, Y, "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Average change\nin Total Surface Energy\nacross the ensemble")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_01.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(MAIN["data"]["tri"])
Y = np.copy(MAIN["data"]["concentration"])

datasets = [
    ("Data", X, Y, "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Value of\nEnsemble [ Concentration ]")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_02.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(MAIN["data"]["tri"])
Y = np.copy(CFG["N_tot"] - MAIN["data"]["n_gas"])
datasets = [
    ("Data", X, Y, "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Value of\nEnsemble [ Ntotal - Ngas ]")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_03.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = SUB2["tri"]
Y = SUB2["n_crystal"]

XY = []

for (t1, t2) in zip(X, Y):
    mask = (t1 != None) & (t2 != None)
    if True in mask:
        XY.append((np.mean(t1[mask]), t2[mask].size))

X, Y = np.array(XY).T

datasets = [
    ("Data", X, Y, "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Value of\nEnsemble [ Alive Particles ]")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_04.png", format="png")
plt.show()
plt.close()

In [ ]:
tri = SUB2["tri"]
Sx = SUB2["crystal_sx"]
Sy = SUB2["crystal_sy"]
Sz = SUB2["crystal_sz"]

X = []
Sxyz_mean = []
Sxyz_STD = []
Sxyz2_mean = []
Sxyz2_STD = []
Sxyz3_mean = []
Sxyz3_STD = []

for t1, t2, t3, t4 in zip(tri, Sx, Sy, Sz):
    mask = (t1 != None) & (t2 != None) & (t3 != None) & (t4 != None)
    if True in mask:
        X.append(np.mean(t1[mask]))

        Sx_items, Sy_items, Sz_items = t2[mask], t3[mask], t4[mask]
        Sx_items, Sy_items, Sz_items
        Sx2_items, Sy2_items, Sz2_items = (
            Sx_items**2.0,
            Sy_items**2.0,
            Sz_items**2.0,
        )
        Sx3_items, Sy3_items, Sz3_items = (
            Sx_items**3.0,
            Sy_items**3.0,
            Sz_items**3.0,
        )

        Sxyz_mean.append((np.mean(Sx_items), np.mean(Sy_items), np.mean(Sz_items)))
        Sxyz_STD.append(
            (
                np.std(Sx_items, ddof=1),
                np.std(Sy_items, ddof=1),
                np.std(Sz_items, ddof=1),
            )
        )

        Sxyz2_mean.append((np.mean(Sx2_items), np.mean(Sy2_items), np.mean(Sz2_items)))
        Sxyz2_STD.append(
            (
                np.std(Sx2_items, ddof=1),
                np.std(Sy2_items, ddof=1),
                np.std(Sz2_items, ddof=1),
            )
        )

        Sxyz3_mean.append((np.mean(Sx3_items), np.mean(Sy3_items), np.mean(Sz3_items)))
        Sxyz3_STD.append(
            (
                np.std(Sx3_items, ddof=1),
                np.std(Sy3_items, ddof=1),
                np.std(Sz3_items, ddof=1),
            )
        )

X = np.array(X)

(Sx_mean, Sy_mean, Sz_mean) = np.array(Sxyz_mean).T
(Sx_STD, Sy_STD, Sz_STD) = np.array(Sxyz_STD).T

(Sx2_mean, Sy2_mean, Sz2_mean) = np.array(Sxyz2_mean).T
(Sx2_STD, Sy2_STD, Sz2_STD) = np.array(Sxyz2_STD).T

(Sx3_mean, Sy3_mean, Sz3_mean) = np.array(Sxyz3_mean).T
(Sx3_STD, Sy3_STD, Sz3_STD) = np.array(Sxyz3_STD).T

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

datasets = [
    (f"X axis", X, Sx_mean, "solid", 0.60, "s", ".50"),
    (f"Y axis", X, Sy_mean, "solid", 0.60, "o", ".25"),
    (f"Z axis", X, Sz_mean, "solid", 0.60, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Average axial particle size\nin the Ensemble")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_05_1.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

datasets = [
    (f"X axis", X, Sx_STD, "solid", 0.60, "s", ".50"),
    (f"Y axis", X, Sy_STD, "solid", 0.60, "o", ".25"),
    (f"Z axis", X, Sz_STD, "solid", 0.60, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("STD [ axial particle size ]\nin the Ensemble")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_05_2.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

datasets = [
    (f"X axis", X, Sx2_mean, "solid", 0.60, "s", ".50"),
    (f"Y axis", X, Sy2_mean, "solid", 0.60, "o", ".25"),
    (f"Z axis", X, Sz2_mean, "solid", 0.60, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Average [ (axial particle size)^2 ]\nin the Ensemble")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_06_1.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

datasets = [
    (f"X axis", X, Sx2_STD, "solid", 0.60, "s", ".50"),
    (f"Y axis", X, Sy2_STD, "solid", 0.60, "o", ".25"),
    (f"Z axis", X, Sz2_STD, "solid", 0.60, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("STD [ (axial particle size)^2 ]\nin the Ensemble")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_06_2.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

datasets = [
    (f"X axis", X, Sx3_mean, "solid", 0.60, "s", ".50"),
    (f"Y axis", X, Sy3_mean, "solid", 0.60, "o", ".25"),
    (f"Z axis", X, Sz3_mean, "solid", 0.60, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Average [ (axial particle size)^3 ]\nin the Ensemble")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_07_1.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

datasets = [
    (f"X axis", X, Sx3_STD, "solid", 0.60, "s", ".50"),
    (f"Y axis", X, Sy3_STD, "solid", 0.60, "o", ".25"),
    (f"Z axis", X, Sz3_STD, "solid", 0.60, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("STD [ (axial particle size)^3 ]\nin the Ensemble")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_07_2.png", format="png")
plt.show()
plt.close()